In [12]:
#load in the used packages
import pandas as pd
import numpy as np
import requests, sys
from collections import Counter

In [13]:
# load the needed databases or donload them
kegg_server = "http://rest.kegg.jp/"
kegg_protein_list = "list/hsa"

kegg_request = requests.get(kegg_server +kegg_protein_list)
if not kegg_request.ok:
    kegg_request.raise_for_status()
    sys.exit()
ensmbl_to_kegg = pd.DataFrame([x.split("\t") for x in kegg_request.text.split("\n")], columns=["ID","names"])
ensmbl_to_kegg = ensmbl_to_kegg.set_index("ID").dropna()
ensmbl_to_kegg = ensmbl_to_kegg.names.apply(lambda x: x.split("; ")[0])
ensmbl_to_kegg = ensmbl_to_kegg.apply(lambda x: x.split(", "))


kegg_server = "http://rest.kegg.jp/"
kegg_protein_pathway = "/link/hsa/pathway"

pathway_request = requests.get(kegg_server +kegg_protein_pathway)
if not pathway_request.ok:
    pathway_request.raise_for_status()
    sys.exit()
path_df = pd.DataFrame([x.split("\t") for x in pathway_request.text.split("\n")], columns=["pathway", "gene_id"])


In [14]:
#Function to retrieve the pathways for each part of the a chromosome.
col_names = ['end', 'biotype', 'strand', 'feature_type', 'id', 'version', 'seq_region_name', 'start',
             'logic_name', 'assembly_name', 'description', 'gene_id', 'source', 'external_name']
def get_pathways(Ch,start,end):
    kegg_id = []
    server = "https://rest.ensembl.org"
    ext = "/overlap/region/human/{0}:{1}-{2}?feature=gene;biotype=protein_coding;a=ncbi36".format(Ch, start,end)
    r = requests.get(server +ext, headers={ "Content-Type" : "application/json"})
    if not r.ok:
        print(f"Error at CH {Ch} from {start} till {end}")
        return []
    #print(pd.DataFrame(r.json()))
    genes = pd.DataFrame(r.json(), columns=col_names).external_name
    for id in list(genes):
        kegg_id += list(ensmbl_to_kegg[ensmbl_to_kegg.map(set([id]).issubset)].index)
    data = path_df[path_df.gene_id.apply(lambda x: x in kegg_id)].pathway.tolist()
    return data

In [15]:
#load the data:
data = pd.read_csv("train_call.txt",sep="\t")
classes = pd.read_csv("train_clinical.txt", sep="\t", index_col=0)


In [16]:
# get chomosoon info
topInfo = data.transpose()[:4]
topInfo = topInfo.astype(int)
topInfo = topInfo.transpose()


In [23]:
pathway_cluster = pd.DataFrame(columns=path_df.pathway.unique()[:-1])
max_seq_range = 5000000-1
for i in topInfo.index:
    full_data = []
    dataPoint = topInfo.iloc[i]
    #print(dataPoint[1],dataPoint[2] )
    Ch = dataPoint[0]
    seq_range = range(np.math.ceil((dataPoint[2] - dataPoint[1])// max_seq_range)+1)
    for x in seq_range:
        if x != seq_range[-1]:
            start = dataPoint[1]+(max_seq_range*x)
            end = dataPoint[1]+(max_seq_range*(x+1))
        elif x == seq_range[-1]:
            start = dataPoint[1]+(max_seq_range*x)
            end = dataPoint[1]+(max_seq_range*(x))+(dataPoint[2] - dataPoint[1])% max_seq_range
        data_test = get_pathways(Ch, start, end)
        #print(len(data_test), len(full_data))
        full_data+=data_test

    pathway_cluster = pathway_cluster.append(Counter(full_data), ignore_index=True)
    #print(pathway_cluster)

Error at CH 2 from 242430070 till 242784096
Error at CH 3 from 199333505 till 199383479
Error at CH 4 from 190887518 till 191025356
Error at CH 4 from 191037678 till 191046020
Error at CH 4 from 191062719 till 191134340
Error at CH 4 from 191144773 till 191165971
Error at CH 4 from 191191940 till 191311992
Error at CH 6 from 170856656 till 170943103
Error at CH 8 from 145962567 till 146264846


In [116]:
#dotprosuct the data to retrieve the sumdata per
pathway_data = pathway_cluster.iloc[:-1].fillna(0).transpose() @ data.dropna().iloc[:,4:].to_numpy()
final_data = pd.concat([classes.reset_index().Subgroup, pathway_data.transpose()], ignore_index=True, axis=1)
final_data.columns = pathway_cluster.columns.insert(0,"Subgroup")
final_data= final_data.set_index("Subgroup")

In [121]:
final_data.groupby("Subgroup").mean()

,path:hsa00010,path:hsa00020,path:hsa00030,path:hsa00040,path:hsa00051,path:hsa00052,path:hsa00053,path:hsa00061,path:hsa00062,path:hsa00071,...,path:hsa05330,path:hsa05332,path:hsa05340,path:hsa05410,path:hsa05412,path:hsa05414,path:hsa05415,path:hsa05416,path:hsa05417,path:hsa05418
Subgroup,,,,,,,,,,,,,,,,,,,,,
HER2+,-0.031250,0.531250,-0.656250,-0.656250,2.031250,0.125000,-1.562500,-1.156250,0.375000,-2.156250,...,-0.312500,0.250000,2.437500,4.906250,1.500000,4.750000,5.593750,-0.156250,1.406250,2.625
HR+,1.305556,1.305556,-0.611111,-0.694444,1.388889,0.138889,-1.194444,-0.333333,0.416667,-0.638889,...,0.305556,-0.361111,2.138889,6.222222,2.305556,5.166667,9.055556,-0.805556,2.388889,3.750
Triple Neg,0.968750,1.656250,-0.343750,0.531250,2.750000,2.406250,-0.781250,-0.906250,2.312500,-1.468750,...,4.718750,5.687500,4.000000,7.750000,3.906250,6.968750,10.593750,4.437500,9.250000,10.125
